# Clustering

In [4]:
import pandas as pd
import os
import numpy as np 
from sklearn.cluster import KMeans
import h5py
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import numpy as np
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import torch
import random
import matplotlib as mpl
font = {'family' : 'Calibri',
        'weight' : 'light',
        'size'   : 12}

mpl.rc('font', **font)


In [5]:
data_path = r''

In [6]:
results = os.listdir(data_path)
all_df = []

for _scan in results:
    df = pd.read_csv(os.path.join(data_path,_scan))
    df = df.dropna()
    df['UniqueIdentifier'] = _scan[:-4]
    df = df.rename(columns={'Unnamed: 0':'frame'})
    
    all_df.append(df)

all_df = pd.concat(all_df)


FileNotFoundError: [WinError 3] The system cannot find the path specified: ''

In [ ]:
all_df = all_df.sample(frac=1)

In [ ]:
data_df = df.drop(['frame','UniqueIdentifier'],axis=1)


In [ ]:
vectors = data_df.to_numpy()

### Elbow method

In [ ]:
from scipy.spatial.distance import cdist

distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 30)
  
for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(vectors)
    kmeanModel.fit(vectors)
  
    distortions.append(sum(np.min(cdist(vectors, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / vectors.shape[0])
    inertias.append(kmeanModel.inertia_)
  
    mapping1[k] = sum(np.min(cdist(vectors, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / vectors.shape[0]
    mapping2[k] = kmeanModel.inertia_

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('Elbow Method for output size of 128')
plt.show()

### View mapping to clusters on the training set

In [ ]:
save_path = r''
image_path = r''

npy_files = os.listdir(image_path)
random.shuffle(npy_files)

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(vectors)

In [ ]:


for npy_file in npy_files:
    print(npy_file)
    im = np.load(os.path.join(image_path,npy_file))
    number_idx = npy_file.rfind('_')
    scan = npy_file[:number_idx]
    frame = int(npy_file[number_idx+1:-4])

    vector = all_df[(all_df.frame==frame)&(all_df.UniqueIdentifier==scan)].drop(['frame','UniqueIdentifier'],axis=1).to_numpy()#.astype('float32')
    if len(vector)>0:
        cluster = kmeans.predict(vector)[0]
        cluster_save_path = save_path + '\\' + str(cluster)

        if not os.path.isdir(cluster_save_path):
            os.mkdir(cluster_save_path)

        plt.imshow(im,cmap = 'gray')
        plt.axis('off')
        plt.savefig(os.path.join(cluster_save_path,'{}_{}.png'.format(scan,frame)))



### Evaluate clustering on a labeled test set

In [ ]:

Model_path = r'C:\Users\ShelleyJoyLevy\Documents\similarity\09_27_22_deep_stride_64'
os.chdir(Model_path)
from SimilarityModelDeepStride64 import SimilarityNetwork,TripletLoss


net = SimilarityNetwork()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net.load_state_dict(torch.load(Model_path + '\BestDev.p',map_location=torch.device(device)))
net.eval();
data_path = r'\\nv-nas01\Algorithm\python\codes\semantic_segmentation\Data\Hoct_label_manual\labeld\final\Asclepix_AMD\Labeld_Data'
mats = os.listdir(data_path)

In [ ]:
result_df = []
save_path = r''

for matname in mats:
    curr_df = {}
    mat = sio.loadmat(os.path.join(data_path,matname))
    im = mat['I'].astype('float32')
    curr_df['with_SRF'] = 1 in np.unique(mat['Ilab'])
    curr_df['with_IRF'] = 2 in np.unique(mat['Ilab'])
    curr_df['wet'] = (curr_df['with_SRF']) or (curr_df['with_IRF']) #or (curr_df['with_ERM'])
    curr_df['dry'] = not curr_df['wet']

    curr_df['fname'] = matname[:-4]
    I_tensor = torch.from_numpy(im)
    I_predict = torch.from_numpy(np.expand_dims(I_tensor, (0,1)))
    score = np.squeeze(net(I_predict).detach().numpy()).reshape(1, -1).astype('float64')
    cluster = kmeans.predict(score)[0]
    curr_df['cluster'] = cluster
    result_df.append(curr_df)
    print(matname,cluster,curr_df['with_SRF'],curr_df['with_IRF'])
    cluster_save_path = save_path + '\\' + str(cluster)

    

In [ ]:
result_df = pd.DataFrame(result_df)

In [ ]:
result_df.groupby('cluster').sum()